<a href="https://colab.research.google.com/github/Raghad-Odwan/Dataset-For-Project/blob/main/Dataset_DermAI_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **DermAI** — Dataset Cleaning & Preprocessing

#### Uploading the dataset from Google Drive and extracted Dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# extracted Dataset
import zipfile
import os

zip_path = "/content/drive/MyDrive/Dataset.zip"
extract_path = "/content/Dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")
os.listdir(extract_path)

Dataset extracted successfully!


['Dataset']

In [ ]:
dataset_path = "/content/Dataset/Dataset"

for item in os.listdir(dataset_path):
    print(item)

malignant
benign


#### Initialize the environment and import required libraries

In [ ]:
!pip install -q --upgrade "tqdm>=4.67" "opencv-python-headless>=4.9.0.80" "albumentations>=1.3.1"

In [ ]:
# Import libraries
import os
import shutil
import hashlib
import random
import math
from tqdm import tqdm
from PIL import Image, UnidentifiedImageError
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings
from pathlib import Path
from collections import defaultdict
from datetime import datetime

# Ignore warnings
warnings.filterwarnings('ignore')

### **Data Cleaning and Exploration**

In [16]:
base_dir = "/content/Dataset/Dataset"
folders = ["benign", "malignant"]


In [17]:
# Create a folder for corrupted or duplicate images
dup_dir = os.path.join(base_dir, "duplicates_or_corrupted")
os.makedirs(dup_dir, exist_ok=True)

print("Base directory:", base_dir)
print("Duplicate/Corrupted folder created at:", dup_dir)

Base directory: /content/Dataset/Dataset
Duplicate/Corrupted folder created at: /content/Dataset/Dataset/duplicates_or_corrupted


##### Data Cleaning and Quality Verification

In [10]:
class ImageCleaner:
    def __init__(self, base_path):
        self.base_path = Path(base_path)
        self.folders_to_check = ['benign', 'malignant']

        # Create folder for problematic files
        self.problem_folder = self.base_path / 'duplicates_or_corrupted'
        self.problem_folder.mkdir(exist_ok=True)

        # Statistics
        self.stats = {
            'total_checked': 0,
            'corrupted': 0,
            'duplicates': 0,
            'low_quality': 0,
            'healthy': 0
        }

        # Store image hashes
        self.image_hashes = defaultdict(list)

    def calculate_hash(self, image_path):
        try:
            hasher = hashlib.md5()
            with open(image_path, 'rb') as f:
                buf = f.read()
                hasher.update(buf)
            return hasher.hexdigest()
        except Exception as e:
            print(f"Error calculating hash: {image_path} - {e}")
            return None

    def is_image_corrupted(self, image_path):
        try:
            with Image.open(image_path) as img:
                img.verify()

            with Image.open(image_path) as img:
                img.load()

            return False
        except Exception:
            return True

    def check_image_quality(self, image_path, min_width=50, min_height=50):
        try:
            with Image.open(image_path) as img:
                width, height = img.size

                if width < min_width or height < min_height:
                    return False

                file_size = os.path.getsize(image_path)
                if file_size < 1000:
                    return False

            return True
        except Exception:
            return False

    def move_to_problem_folder(self, image_path):
        try:
            source_folder = image_path.parent.name
            dest_subfolder = self.problem_folder / source_folder
            dest_subfolder.mkdir(exist_ok=True)

            destination = dest_subfolder / image_path.name

            if destination.exists():
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                destination = dest_subfolder / f"{image_path.stem}_{timestamp}{image_path.suffix}"

            shutil.move(str(image_path), str(destination))

        except Exception as e:
            print(f"Error moving file: {image_path.name} - {e}")

    def clean_folder(self, folder_name):
        folder_path = self.base_path / folder_name

        if not folder_path.exists():
            print(f"Folder not found: {folder_name}")
            return

        print(f"Processing: {folder_name}...", end=' ')

        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif']
        images = [f for f in folder_path.iterdir()
                 if f.is_file() and f.suffix.lower() in image_extensions]

        for img_path in images:
            self.stats['total_checked'] += 1

            # Check corruption
            if self.is_image_corrupted(img_path):
                self.move_to_problem_folder(img_path)
                self.stats['corrupted'] += 1
                continue

            # Check quality
            if not self.check_image_quality(img_path):
                self.move_to_problem_folder(img_path)
                self.stats['low_quality'] += 1
                continue

            # Check duplicates
            img_hash = self.calculate_hash(img_path)
            if img_hash:
                if img_hash in self.image_hashes:
                    self.move_to_problem_folder(img_path)
                    self.stats['duplicates'] += 1
                else:
                    self.image_hashes[img_hash].append(str(img_path))
                    self.stats['healthy'] += 1

        print(f"Done ({len(images)} images checked)")

    def clean_all(self):
        print("\n" + "="*60)
        print("Starting cleaning process")
        print("="*60)

        for folder in self.folders_to_check:
            self.clean_folder(folder)

        self.print_summary()

    def print_summary(self):
        print("\n" + "="*60)
        print("Cleaning Summary")
        print("="*60)
        print(f"Total checked: {self.stats['total_checked']}")
        print(f"Healthy images: {self.stats['healthy']}")
        print(f"Corrupted: {self.stats['corrupted']}")
        print(f"Duplicates: {self.stats['duplicates']}")
        print(f"Low quality: {self.stats['low_quality']}")
        print(f"\nProblematic files moved to: {self.problem_folder}")
        print("="*60)


# For Google Colab - use the path you already created
base_path = "/content/Dataset/Dataset"

# Check if path exists
if not os.path.exists(base_path):
    print(f"Error: Path does not exist: {base_path}")
    print("Current working directory:", os.getcwd())
    print("\nAvailable folders:")
    if os.path.exists("/content/Dataset"):
        print(os.listdir("/content/Dataset"))
else:
    print(f"Base path found: {base_path}")
    print(f"Contents: {os.listdir(base_path)}")

    cleaner = ImageCleaner(base_path)
    cleaner.clean_all()
    print("\nCleaning completed successfully!")

Base path found: /content/Dataset/Dataset
Contents: ['duplicates_or_corrupted', 'malignant', 'benign']

Starting cleaning process
Processing: benign... Done (13294 images checked)
Processing: malignant... Done (6211 images checked)

Cleaning Summary
Total checked: 19505
Healthy images: 19505
Corrupted: 0
Duplicates: 0
Low quality: 0

Problematic files moved to: /content/Dataset/Dataset/duplicates_or_corrupted

Cleaning completed successfully!


##### Data preprocessing (resizing, split)

In [11]:
import cv2
import shutil
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
BASE_DIR = "/content/Dataset/Dataset"
FOLDERS = ["benign", "malignant"]
IMG_SIZE = (224, 224)
RANDOM_STATE = 42

TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15
SPLIT_ROOT = os.path.join(BASE_DIR)

In [15]:
# Resize all images to 224x224 and overwrite the originals as JPG files
for cat in FOLDERS:
    src_dir = os.path.join(BASE_DIR, cat)
    files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]

    for fname in tqdm(files, desc=f"Resizing {cat}"):
        src_path = os.path.join(src_dir, fname)
        try:
            if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue


            # read image
            img = cv2.imread(src_path)
            if img is None:
                print(f" Skipped unreadable file: {fname}")
                continue

            # resize image
            resized = cv2.resize(img, IMG_SIZE, interpolation=cv2.INTER_AREA)

            # save resized image as JPG (overwrite or replace)
            base_name = os.path.splitext(fname)[0]
            save_name = base_name + ".jpg"
            save_path = os.path.join(src_dir, save_name)
            cv2.imwrite(save_path, resized)

            # remove original file if extension changed
            if save_path != src_path:
                os.remove(src_path)

        except Exception as e:
            print(f" Skipped {fname} due to error: {e}")

Resizing malignant: 100%|██████████| 6211/6211 [00:04<00:00, 1540.25it/s]


**Split dataset: 70% train, 15% val, 15% test**


In [18]:
import os, shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [19]:
base_dir = "/content/Dataset/Dataset"
folders = ["benign", "malignant"]

split_dir = "/content/Dataset_split"
os.makedirs(split_dir, exist_ok=True)


# Collect all image file paths and labels
rows = []
for label in folders:
    path = os.path.join(base_dir, label)
    for fname in os.listdir(path):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            rows.append({'path': os.path.join(path, fname), 'label': label})

df = pd.DataFrame(rows)
print("Total images found:", len(df))
print(df['label'].value_counts())

# Stratified split: 70% train, 15% val, 15% test
train_temp, test = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=42)
train, val = train_test_split(train_temp, test_size=0.1765, stratify=train_temp['label'], random_state=42)
# 0.1765 of 85% ≈ 15% of total, resulting in 70/15/15 split

# Create destination folders for train, val, and test
for subset in ['train', 'val', 'test']:
    for label in folders:
        os.makedirs(os.path.join(split_dir, subset, label), exist_ok=True)

# Copy images to their corresponding subset folders
def copy_images(df_subset, subset_name):
    print(f"\nCopying {subset_name} set ({len(df_subset)} images)...")
    for _, row in tqdm(df_subset.iterrows(), total=len(df_subset)):
        dest = os.path.join(split_dir, subset_name, row['label'], os.path.basename(row['path']))
        shutil.copy2(row['path'], dest)

copy_images(train, "train")
copy_images(val, "val")
copy_images(test, "test")

# Print summary
print("\n Dataset split completed successfully!")
print(f"Train: {len(train)} images")
print(f"Val:   {len(val)} images")
print(f"Test:  {len(test)} images")

print("\nClass distribution per set:")
print("Train:\n", train['label'].value_counts())
print("\nVal:\n", val['label'].value_counts())
print("\nTest:\n", test['label'].value_counts())

Total images found: 19505
label
benign       13294
malignant     6211
Name: count, dtype: int64

Copying train set (13652 images)...


100%|██████████| 13652/13652 [00:02<00:00, 5027.63it/s]



Copying val set (2927 images)...


100%|██████████| 2927/2927 [00:00<00:00, 4866.01it/s]



Copying test set (2926 images)...


100%|██████████| 2926/2926 [00:00<00:00, 4929.41it/s]


 Dataset split completed successfully!
Train: 13652 images
Val:   2927 images
Test:  2926 images

Class distribution per set:
Train:
 label
benign       9305
malignant    4347
Name: count, dtype: int64

Val:
 label
benign       1995
malignant     932
Name: count, dtype: int64

Test:
 label
benign       1994
malignant     932
Name: count, dtype: int64
